# Way one: **Threads**

## Begin with fibonacci

In [1]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

## Timing

In [2]:
%%time
fibonacci(34)

CPU times: user 2.38 s, sys: 0 ns, total: 2.38 s
Wall time: 2.39 s


5702887

## Run it twice

In [3]:
def super_expensive(n):
    fibonacci(n)
    fibonacci(n)
    
%time super_expensive(34)

CPU times: user 4.83 s, sys: 0 ns, total: 4.83 s
Wall time: 4.89 s


## Speed it up with threads

Threads are easy.  Python threads are OS threads, and they switch preemptively

```python
import threading

my_thread = threading.Thread(target=my_function)
my_thread.start()

# wait for the thread to finish
my_thread.join()

def my_function():
    ...
```

In [4]:
import threading

def super_expensive(n):
    thread_one = threading.Thread(target=fibonacci,args=(n,))
    thread_one.start()
    thread_two = threading.Thread(target=fibonacci,args=(n,))
    thread_two.start()
    
    thread_one.join()
    thread_two.join()
    
%time super_expensive(34)

CPU times: user 4.81 s, sys: 50 ms, total: 4.86 s
Wall time: 4.91 s


## What's going on? Why doesn't it take 50% of that time?

Maybe threads are sequential....  let's test it


In [ ]:
import sys
import threading

def fibonacci(n, message):
    sys.stdout.write(message)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1, message) + fibonacci(n - 2, message)
    
def super_expensive(n):
    thread_one = threading.Thread(target=fibonacci,args=(n,"_"))
    thread_one.start()
    thread_two = threading.Thread(target=fibonacci,args=(n,"*"))
    thread_two.start()
    
    thread_one.join()
    thread_two.join()  
    
%time super_expensive(15)

# Plot thickener: Amdahl's law
![amdahl](images/Amdahl.png)

# Insight: An implementation of the python language is, itself, a multithreaded program

Thread one:
```python
global_str = "abcdefg"
```

Thread two:
```python
global_str = "hijklmnop"
```

The interpreter must ensure that global_str is one value or the other and NOT a hybrid of the two
```python
print(global_str)
abclmnop
```

# Meet GIL

<img src="images/villian.svg" alt="drawing" width="500"/>

## What does GIL do?

You **have** to have locks somewhere in the python interpreter to keep the interpreter state rational in the face of threading.   C-Python opted for a single global lock rather than dispersed, finer grained locks

1. Gil limits access to the interpreter to one thread at a time.
2. This means that only one thread at a time can execute python code in the interpreter.
3. You will have access to CONCURRENCY but not to PARALLELISM when using Python threads.

**You have the net effect of using a SINGLE core when using threads in Python.  Each thread IS an OS thread, may execute on various cores, but only one thread at a time will execute python code**




# An alternate view of Gil
<img src="images/gill.jpg" alt="drawing" width="1000"/>


## Be Aware of Gil, but don't Beware of Gil

## And remember that concurrency requires thread synchronization

Locking
```python
import threading

my_lock = threading.Lock()

def worker():
    with my_lock:
        # protected stuff
```

**There are many additional thread synchronization constructs you can investigate**
* RLock
* Semaphore
* Event
* Condition
* Barrier
* ...

## Give yourself some sanity.

1. Access global state from ONE thread only
2. Communicate to that ONE thread from other threads using a Queue
3. To wait for thread(s), join() the thread or the Queue to signal no work is left
4. If you violate 1-3, use thread local variables if at all possible



In [ ]:
# A worked example from Raymond Hettinger

import threading, queue

counter = 0
counter_queue = queue.Queue()

def counter_manager():
    'I have EXCLUSIVE rights to update the counter variable'
    global counter

    while True:
        increment = counter_queue.get()
        counter += increment
        print_queue.put([
            'The count is %d' % counter,
            '---------------'])
        counter_queue.task_done()

t = threading.Thread(target=counter_manager)
t.daemon = True
t.start()
del t

print_queue = queue.Queue()

def print_manager():
    'I have EXCLUSIVE rights to call the "print" keyword'
    while True:
        job = print_queue.get()
        for line in job:
            print(line)
        print_queue.task_done()

t = threading.Thread(target=print_manager)
t.daemon = True
t.start()
del t


def worker():
    'My job is to increment the counter and print the current count'
    counter_queue.put(1)

print_queue.put(['Starting up'])
worker_threads = []
for i in range(10):
    t = threading.Thread(target=worker)
    worker_threads.append(t)
    t.start()
for t in worker_threads:
    t.join()

counter_queue.join()
print_queue.put(['Finishing up'])
print_queue.join()

# Lessons learned

1. Threads are easy, threading is it's usual self
2. Threads are OS threads and switch pre-emptively
3. Python threads are limited to, effectively, a single core
4.  &#9670; &#9670; Threads are a low-level concept.  Use higher level APIs if possible (e.g. concurrent.futures) 
5. When programming with threads, use concurrency patterns to help yourself out:
    1. Access global state from ONE thread only
    2. Communicate to that ONE thread from other threads using a Queue
    3. To wait for thread(s), join() the thread or the Queue to signal no work is left
    4. If you violate 1-3, use thread local variables if at all possible



### Footnote: Python, the language, does not require a GIL.

C-Python - the implementation of python you are most likely to use - does have a GIL

--> Mostly because of the memory manager

--> Removing the GIL has been done many times